### 1. 模拟在不同adversary比例下Gearbox全部shard的平均size和overlaptime
- 总节点数量：5000
- Gearbox: 10%, 20%, 25% 30% 和50%
- 对应的 shard size: 26,39,50,63,293

In [18]:
import random

"""The simulation about Gearbox:
    We generated 5000 nodes and there are five gears, gears size are 18,29,38,46,221. We calculate it from the Shard size.ipynb with failure rate fall in 10^-7 and 10^-8.
    We assign nodes into shard randomly and check if the adversarial nodes in the shard is more than this shard's livness threshold or we turn to a bigger one.
    we keep doing this until all the shard can run normally and then calculate the average shard size. Also,we calculate the overlap time.
"""

"""Function run_simulation used for find the average shard size of Gearbox in different adversarial nodes ratio.
    Input:
        n: The adversarial nodes ratio in the system(upper bound).
        
    Output:
        average shard size of all shard
        
"""

# Function run_simulation used for finding the average shard size of Gearbox with different adversarial nodes ratio.
def run_simulation(n):
    # Calculate the number of Byzantine nodes based on the percentage 'n'
    byzantine_nodes = int(n / 100 * 5000)
    results = []
    element_values = []  # 用于存储每次运行的 element 值
    for i in range(192):
        # Step 1: Select a sample of 18 nodes
        sample_1 = random.sample(range(5000), 26)
        num_byzantine_nodes = sum(1 for node in sample_1 if node < byzantine_nodes)
        if num_byzantine_nodes > 2:
            sample_2 = random.sample(range(5000), 39)
            num_byzantine_nodes = sum(1 for node in sample_2 if node < byzantine_nodes)
            if num_byzantine_nodes > 7:
                sample_3 = random.sample(range(5000), 50)
                num_byzantine_nodes = sum(1 for node in sample_3 if node < byzantine_nodes)
                if num_byzantine_nodes > 12:
                    sample_4 = random.sample(range(5000), 63)
                    num_byzantine_nodes = sum(1 for node in sample_4 if node < byzantine_nodes)
                    if num_byzantine_nodes > 18:
                        element = 293
                    else:
                        element = 63
                else:
                    element = 50
            else:
                element = 39
        else:
            element = 26
        
        results.append(element)
        element_values.append(element)  # 收集每次运行的 element 值

    # Calculate the average number of records
    return sum(results) / len(results), element_values

gresult = []
goverlap = []

for n in range(50):
    results = []
    element_results = []

    for i in range(1000):
        # Run the simulation for 'n' and obtain the average number of records and element values
        avg_records, element_values = run_simulation(n)
        results.append(avg_records)
        element_results.extend(element_values)  # 将每次运行的 element 值添加到 element_results

    # 计算每个元素的百分比
    total_elements = len(element_results)
    element_percentages = {26: 0, 39: 0, 50: 0, 63: 0, 293: 0}

    for element in element_results:
        element_percentages[element] += 1

    # 将百分比除以总数以确保总和为100%
    for key in element_percentages:
        element_percentages[key] /= total_elements

    # 输出每个元素的百分比
    print(f"n={n}:")
    for key, percentage in element_percentages.items():
        print(f"{key}: {percentage * 100:.2f}%")

    gresult.append(sum(results) / len(results))
    print(f"n={n}: {gresult[-1]}")
    # Calculate the overlap time by dividing the average results by 18
    print(f"overlap time: {gresult[-1] / 26}")
    goverlap.append(gresult[-1] / 26)


n=0:
26: 100.00%
39: 0.00%
50: 0.00%
63: 0.00%
293: 0.00%
n=0: 26.0
overlap time: 1.0
n=1:
26: 99.79%
39: 0.21%
50: 0.00%
63: 0.00%
293: 0.00%
n=1: 26.027218749999935
overlap time: 1.0010468749999974
n=2:
26: 98.52%
39: 1.48%
50: 0.00%
63: 0.00%
293: 0.00%
n=2: 26.19229166666668
overlap time: 1.007395833333334
n=3:
26: 95.82%
39: 4.18%
50: 0.00%
63: 0.00%
293: 0.00%
n=3: 26.543359374999998
overlap time: 1.0208984374999999
n=4:
26: 91.64%
39: 8.36%
50: 0.00%
63: 0.00%
293: 0.00%
n=4: 27.08737500000001
overlap time: 1.0418221153846157
n=5:
26: 86.19%
39: 13.80%
50: 0.01%
63: 0.00%
293: 0.00%
n=5: 27.797041666666694
overlap time: 1.069116987179488
n=6:
26: 79.60%
39: 20.36%
50: 0.04%
63: 0.00%
293: 0.00%
n=6: 28.656145833333355
overlap time: 1.102159455128206
n=7:
26: 72.85%
39: 27.01%
50: 0.14%
63: 0.00%
293: 0.00%
n=7: 29.544921875000014
overlap time: 1.1363431490384621
n=8:
26: 65.48%
39: 34.16%
50: 0.35%
63: 0.00%
293: 0.00%
n=8: 30.526010416666697
overlap time: 1.1740773237179498
n=9

## 2. 拟合出shard size和time-bound的线性关系
- 类似于 Reticulum，Gearbox 同样包含两条链，因此我们粗略的使用Reticulum的 time-bound进行拟合
- ($N_c$,886), ($N_p$,136)

In [1]:
"""This function is try to get a fitted equation about connected nodes number and """

import random
import matplotlib.pyplot as plt
import numpy as np

# Define the input data
x = np.array([329])  # Input x values
y = np.array([689])  # Input y values

# Fit a linear equation (one degree polynomial) to the data
coefficients = np.polyfit(x, y, 1)

# Extract the slope and intercept from the coefficients
slope = coefficients[0]
intercept = coefficients[1]

# Print the equation of the fitted line
print(f"The fitted equation is: y = {slope:.2f}x + {intercept:.2f}")

The fitted equation is: y = 1.05x + 344.50


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


## 3.计算 Gearbox在不同adversary比例下的吞吐量
- 假设每个非最小Gear的shard都尝试在新的epoch下调小gear

### 3.1 当 $P_a=10\%$

n=10:\
26: 50.78%\
39: 47.50%\
50: 1.72%\
63: 0.00%\
293: 0.00%\
n=10: 32.588233333333356\
overlap time: 1.2533935897435906

- 50.78%的shard的共识是在26个节点内达成的，他们花费的时间是：26 * 2.44+84.86
- 47.50%的shard的共识是在39个节点内达成的，他们花费的时间是：26 * 2.44+84.86+39*2.44+84.86
- 1.72%的shard的共识是在50个节点内达成的，他们花费的时间是：39 * 2.44+84.86+50*2.44+84.86

- 总平均共识达成时间:248.324s


- 吞吐量= (5000/26)*4096 /248.324 /1.253 = 2527.5 tx/sec


- 50.78%的shard的共识是在26个节点内达成的，他们花费的时间是：26*2.44+84.86
- 47.50%的shard的共识是在39个节点内达成的，他们花费的时间是：39*2.44+84.86
- 1.72%的shard的共识是在50个节点内达成的，他们花费的时间是：50*2.44+84.86

- 总平均共识达成时间:164.37s


- 吞吐量= (5000/26)*4096 /164.37 /1.253 = 3824.6 tx/sec

In [12]:
(5000/26)*4096 /106 /1.253

5930.6141313098215

In [11]:
50*2.12

106.0

### 3.2 当 $P_a=20\%$

n=20:\
26: 8.32%\
39: 42.41%\
50: 40.27%\
63: 8.63%\
293: 0.37%\
n=20: 45.35866666666669\
overlap time: 1.7445641025641037

- 8.32%的shard的共识是在26个节点内达成的，他们花费的时间是：26*2.44+84.86
- 42.41%的shard的共识是在39个节点内达成的，他们花费的时间是：26*2.44+84.86+39*2.44+84.86
- 40.27%的shard的共识是在50个节点内达成的，他们花费的时间是：39*2.44+84.86+50*2.44+84.86
- 8.63%的shard的共识是在63个节点内达成的，他们花费的时间是：50*2.44+84.86+63*2.44+84.86
- 0.37%的shard的共识是在63个节点内达成的，他们花费的时间是：293*2.44+84.86+63*2.44+84.86

- 总平均共识达成时间:311.218s

- 吞吐量= (5000/26)*4096 /311.218 /1.744 = 1451.3 tx/sec

- 8.32%的shard的共识是在26个节点内达成的，他们花费的时间是：26*2.44+84.86
- 42.41%的shard的共识是在39个节点内达成的，他们花费的时间是：39*2.44+84.86
- 40.27%的shard的共识是在50个节点内达成的，他们花费的时间是：50*2.44+84.86
- 8.63%的shard的共识是在63个节点内达成的，他们花费的时间是：63*2.44+84.86
- 0.37%的shard的共识是在63个节点内达成的，他们花费的时间是：293*2.44+84.86

- 总平均共识达成时间:177.01s

- 吞吐量= (5000/26)*4096 /177.01 /1.744 = 2551.59tx/sec

In [21]:
0.0832*(26*2.44+84.86)+0.4241*(39*2.44+84.86)+0.4027*(50*2.44+84.86)+0.00863*(63*2.44+84.86)+0.0037*(293*2.44+84.86)

177.0056954

In [13]:
63*2.12

133.56

In [14]:
(5000/26)*4096 /133.56 /1.744

3381.689377881173

### 3.3 当 $P_a=30\%$

n=30:\
26: 0.66%\
39: 6.53%\
50: 20.38%\
63: 33.35%\
293: 39.08%\
n=30: 148.4329000000002\
overlap time: 5.7089576923077


- 0.66%的shard的共识是在26个节点内达成的，他们花费的时间是：26*2.44+84.86
- 6.53%的shard的共识是在39个节点内达成的，他们花费的时间是：39*2.44+84.86
- 20.38%的shard的共识是在50个节点内达成的，他们花费的时间是：50*2.44+84.86
- 33.35%的shard的共识是在63个节点内达成的，他们花费的时间是：63*2.44+84.86
- 39.08%的shard的共识是在293个节点内达成的，他们花费的时间是：293*2.44+84.86

- 总平均共识达成时间:447.013s

- 吞吐量= (5000/26)*4096 /655.610 /5.708 = 308.71 tx/sec

In [23]:
0.0066*(26*2.44+84.86)+0.0653*(39*2.44+84.86)+0.2038*(50*2.44+84.86)+0.3335*(63*2.44+84.86)+0.3908*(293*2.44+84.86)

447.012608

In [16]:
(5000/26)*4096 /621.16 /5.708

222.16168393474476

In [15]:
293*2.12

621.1600000000001